In [11]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, 
    Dropout, BatchNormalization, Input, 
    LeakyReLU, RandomFlip, RandomRotation, 
    RandomZoom, Rescaling
)
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import (
    ModelCheckpoint, 
    EarlyStopping, 
    ReduceLROnPlateau, 
    CSVLogger
)

In [12]:


# Seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)

# Hyperparameters
IMG_HEIGHT = 224  # Increased from original 188
IMG_WIDTH = 224   # Increased from original 188
BATCH_SIZE = 8
EPOCHS = 50       # Increased from original
TEST_SIZE = 0.2
LEARNING_RATE = 1e-4  # Adjusted learning rate
PATIENCE = 15     # Increased patience
L2_RATE = 1e-4    # L2 regularization


In [13]:

# Directories
BASE_DIR = '../'
INPUT_DIR = os.path.join(BASE_DIR, 'input')
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')
MODEL_DIR = os.path.join(BASE_DIR, 'models')
SUBDIR = 'flower_photos'

# Ensure directories exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)


In [14]:

# Load and preprocess dataset
def load_dataset(data_dir):
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        subset="training",
        seed=RANDOM_STATE,
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        validation_split=TEST_SIZE,
    )

    test_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        subset="validation",
        seed=RANDOM_STATE,
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        validation_split=TEST_SIZE,
    )
    
    return train_ds, test_ds


In [16]:

# Load and preprocess dataset
def load_dataset(data_dir):
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        subset="training",
        seed=RANDOM_STATE,
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        validation_split=TEST_SIZE,
    )

    test_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        subset="validation",
        seed=RANDOM_STATE,
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        validation_split=TEST_SIZE,
    )
    
    return train_ds, test_ds


In [17]:

# Performance optimization for dataset
def configure_dataset(ds, augment=False):
    if augment:
        # Data augmentation layers
        data_augmentation = tf.keras.Sequential([
            RandomFlip('horizontal'),
            RandomRotation(0.2),
            RandomZoom(0.2),
        ])
        
        ds = ds.map(
            lambda x, y: (data_augmentation(x, training=True), y), 
            num_parallel_calls=tf.data.AUTOTUNE
        )
    
    return ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)


In [18]:
def create_cnn_model(input_shape, num_classes, l2_rate=1e-4):
    inputs = Input(shape=input_shape)
    
    # Preprocessing
    x = Rescaling(1./255)(inputs)
    
    # First Convolutional Block
    x = Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(l2_rate))(x)
    x = LeakyReLU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)
    
    # Second Convolutional Block
    x = Conv2D(128, (3,3), padding='valid', kernel_regularizer=l2(l2_rate))(x)
    x = LeakyReLU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2), padding='valid')(x)
    x = Dropout(0.3)(x)
    
    # Third Convolutional Block
    x = Conv2D(256, (3,3), padding='valid', kernel_regularizer=l2(l2_rate))(x)
    x = LeakyReLU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2), padding='valid')(x)
    x = Dropout(0.35)(x)
    
    # Fourth Convolutional Block
    x = Conv2D(512, (3,3), padding='valid', kernel_regularizer=l2(l2_rate))(x)
    x = LeakyReLU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2), padding='valid')(x)
    x = Dropout(0.4)(x)
    
    # Fully Connected Layers 
    x = Flatten()(x)
    
    # Head Layers
    x = Dense(512, kernel_regularizer=l2(l2_rate))(x)
    x = LeakyReLU()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    
    x = Dense(128, kernel_regularizer=l2(l2_rate))(x)
    x = LeakyReLU()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    
    # Output Layer
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model


In [19]:

# Main training function
def train_model():
    # Load dataset
    data_dir = os.path.join(INPUT_DIR, SUBDIR)
    train_ds, test_ds = load_dataset(data_dir)
    
    # Get class names
    class_names = train_ds.class_names
    num_classes = len(class_names)
    
    # Configure datasets
    train_ds = configure_dataset(train_ds, augment=True)
    test_ds = configure_dataset(test_ds)
    
    # Create model
    model = create_cnn_model(
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), 
        num_classes=num_classes
    )
    
    # Model Summary
    model.summary()
    
    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Callbacks
    # checkpoint_path = os.path.join(MODEL_DIR, SUBDIR, 'best_custom_model.keras')
    checkpoint_path = os.path.normpath(os.path.join(MODEL_DIR, SUBDIR, 'best_custom_model.keras'))

    os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)
    
    model_checkpoint = ModelCheckpoint(
        checkpoint_path,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    )
    
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=PATIENCE,
        restore_best_weights=True,
        verbose=1
    )
    
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=8,
        min_lr=1e-6,
        verbose=1
    )
    
    csv_logger = CSVLogger(
        os.path.join(OUTPUT_DIR, 'custom_cnn_training_log.csv'), 
        separator=',', 
        append=False
    )
    
    # Train model
    history = model.fit(
        train_ds, 
        epochs=EPOCHS, 
        validation_data=test_ds,
        callbacks=[
            model_checkpoint, 
            early_stopping, 
            reduce_lr,
            csv_logger
        ]
    )
    
    # Evaluate model
    test_loss, test_accuracy = model.evaluate(test_ds)
    print(f'Test accuracy: {test_accuracy * 100:.2f}%')
    
    # Plot training history
    plt.figure(figsize=(12, 4))
    
    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'custom_cnn_training_history.png'))
    plt.close()
    
    return model, history


In [20]:

def train_model():
    # Load dataset
    data_dir = os.path.join(INPUT_DIR, SUBDIR)
    train_ds, test_ds = load_dataset(data_dir)
    
    # Get class names
    class_names = train_ds.class_names
    num_classes = len(class_names)
    
    # Configure datasets
    train_ds = configure_dataset(train_ds, augment=True)
    test_ds = configure_dataset(test_ds)
    
    # Create model
    model = create_cnn_model(
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), 
        num_classes=num_classes
    )
    
    # Compile model with careful learning rate
    model.compile(
        optimizer=Adam(learning_rate=1e-3),  # Start with a slightly lower learning rate
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Callbacks with more patience
    checkpoint_path = os.path.normpath(os.path.join(MODEL_DIR, SUBDIR, 'best_improved_model.keras'))
    os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)
    
    model_checkpoint = ModelCheckpoint(
        checkpoint_path,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    )
    
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=15,  # Increased patience
        restore_best_weights=True,
        verbose=1
    )
    
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=7,  # Increased patience for learning rate reduction
        min_lr=1e-6,
        verbose=1
    )
    
    csv_logger = CSVLogger(
        os.path.join(OUTPUT_DIR, 'improved_cnn_training_log.csv'), 
        separator=',', 
        append=False
    )
    
    # Train model
    history = model.fit(
        train_ds, 
        epochs=50,  # Increased max epochs 
        validation_data=test_ds,
        callbacks=[
            model_checkpoint, 
            early_stopping, 
            reduce_lr,
            csv_logger
        ]
    )
    
    # Evaluate model
    test_loss, test_accuracy = model.evaluate(test_ds)
    print(f'Test accuracy: {test_accuracy * 100:.2f}%')
    
    # Visualization code remains the same as in previous implementation
    
    return model, history

In [ ]:
# Run the training
if __name__ == '__main__':
    model, history = train_model() 

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.
Epoch 1/50
367/367 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3736 - loss: 2.1515
Epoch 1: val_accuracy improved from -inf to 0.32698, saving model to ..\models\flower_photos\best_improved_model.keras
367/367 ━━━━━━━━━━━━━━━━━━━━ 432s 1s/step - accuracy: 0.3737 - loss: 2.1510 - val_accuracy: 0.3270 - val_loss: 1.8953 - learning_rate: 0.0010
Epoch 2/50
 60/367 ━━━━━━━━━━━━━━━━━━━━ 5:26 1s/step - accuracy: 0.4354 - loss: 1.8313